# Semana 1 - Aprendizagem Supervisonada
---



## Divisão dos dados e Avaliação

### Importando as bibliotecas

In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



### Importando os dados

In [2]:
data_url = 'https://raw.githubusercontent.com/intelligentagents/aprendizagem-supervisionada/master/data/preprocessing_data.csv'

df = pd.read_csv(data_url)

df.head(10)

,City,Age,Salary,Sex,Acquired
0,Paraiba,42.0,6500.0,M,No
1,Sao Paulo,21.0,4300.0,F,Yes
2,Brasilia,28.0,4900.0,F,No
3,Recife,35.0,5400.0,F,No
4,Maceio,38.0,NaN,M,Yes
5,Sao Paulo,33.0,5200.0,M,Yes
6,Belo Horizonte,NaN,4700.0,F,No
7,Sao Paulo,45.0,7100.0,M,Yes
8,Paraiba,51.0,7500.0,F,No
9,Fortaleza,39.0,6000.0,M,Yes


Descrevendo o dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   City      10 non-null     object 
 1   Age       9 non-null      float64
 2   Salary    9 non-null      float64
 3   Sex       10 non-null     object 
 4   Acquired  10 non-null     object 
dtypes: float64(2), object(3)
memory usage: 528.0+ bytes


Preenchendo os registros nuḿericos que não possuem valores com a média.

In [4]:
df = df.fillna(df.mean())

df

/var/folders/96/v98rl5mx1sxdlg9hmsyd9bfm0000gn/T/ipykernel_6353/1988022840.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


,City,Age,Salary,Sex,Acquired
0,Paraiba,42.000000,6500.000000,M,No
1,Sao Paulo,21.000000,4300.000000,F,Yes
2,Brasilia,28.000000,4900.000000,F,No
3,Recife,35.000000,5400.000000,F,No
4,Maceio,38.000000,5733.333333,M,Yes
5,Sao Paulo,33.000000,5200.000000,M,Yes
6,Belo Horizonte,36.888889,4700.000000,F,No
7,Sao Paulo,45.000000,7100.000000,M,Yes
8,Paraiba,51.000000,7500.000000,F,No
9,Fortaleza,39.000000,6000.000000,M,Yes


Definindos as variáveis independentes (X) e dependente (y).

In [5]:
X = df.iloc[:, :-1].values

In [6]:
y = df.iloc[:, 4].values
y

array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes'],
      dtype=object)

## Efetividade na Classificação

Criando as funções que calculam as diferentes métricas de efetividade:

### Cálculo das Medidas de Efetividade

Definindo as funções de efetividade e importando bibliotecas:

In [9]:
# Função que calcula os reais positivos
def rp(tp, fn):
    return tp + fn

# Função que calcula os reais negativos     
def rn(fp, tn):
    return fp + tn

# Função que calcula as predições positivas  
def pp(tp, fp):
    return tp + fp

# Função que calcula as predições negativas   
def pn(fn, tn):
    return fn + tn

# Função que calcula Acurácia do modelo
def accuracy (tp, fp, fn, tn):
     accuracy = ((tp + tn) / (tp + tn + fp + fn))
     return (accuracy)
    
# Função que calcula a Precisão 
def precision (tp, fp):
    precision =  (tp / (tp + fp)) #predições positivas
    return precision

# Função que calcula o Recall ou Taxa de True Positive (TTP)
def recall(tp, fn):
    recall =  (tp / (tp + fn)) # reais positivos
    return recall

## Função que calcula o f-measure (media harmonica entre Precision e Recall)
def f_measure(tp, fp, fn):
    f_measure = (2 * precision(tp, fp) * recall(tp, fn)) / (recall(tp, fn) + precision(tp, fp))
    return f_measure
  
# Função que calcula o Informedness 
def informedness(tp, fp, fn, tn):
    inform = ((tp/rp(tp, fn)) - (fp/rn(fp, tn)))
    return inform

# Função que calcula o Markedness
def markdness(tp, fp, fn, tn):    
    mark = ((tp/pp(tp,fp)) - (fn/pn(fn,tn)))
    return mark

# Função que calcula a taxa de False Positive (TFP)
def tfp(fp, tn):
  tfp = (fp / (tn + fp))
  return tfp

Definido os valores da Matriz de Confusão

In [10]:
tp, fp, fn, tn = [1,1,1,2]

Calculando a Acurácia:

In [11]:
accuracy (tp, fp, fn, tn)

0.6

Calculando a Precision:

In [12]:
precision (tp, fp)

0.5

Calculando o Recall:

In [13]:
 recall(tp, fn)

0.5

Calculando o Informedness: 

In [14]:
informedness(tp, fp, fn, tn)

0.16666666666666669

Calculando o Markedness:

In [15]:
markdness(tp, fp, fn, tn)

0.16666666666666669

## Treinamento e Testes de Modelos de Predição

### Divisão do Dataset

Dividindo o dataset no conjunto de treinamento e testes. Normalmente o conjunto de testes equivale a 1/3 do dataset total:

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 42)

print("Tamanho do Dataset. {}".format(df.shape[0]))
print("Tamanho do Conjunto de Treinamento. {}".format(len(X_train)))
print("Tamanho do Conjunto de Testes. {}".format(len(X_test)))


Tamanho do Dataset. 10
Tamanho do Conjunto de Treinamento. 6
Tamanho do Conjunto de Testes. 4


### Validação Cruzada


Criando uma função para definir os índices de uma validação cruzada:

In [17]:
# Função que retorna os indices do validação cruzada em k folds
def get_folds(indexes, k = 5, seed = 42):
    
    """Função que retorna os indices do validação cruzada em k folds

    Parâmetros
    ----------
    
    indexes : array
        Indices do dataframe
    
    k : int, optional, default=5
        Numero de folds. Deve ser pelo menos 2.
    
    seed : int, optional, default=42
        È o valor usado pelo gerador de números aleatórios;
     """ 
        
    size = len(indexes)
    subset_size = round(size / k)
    random.Random(seed).shuffle(indexes)
    subsets = [indexes[x:x+subset_size] for x in range(0, len(indexes), subset_size)]
    kfolds = []
    for i in range(k):
        test = subsets[i]
        train = []
        for subset in subsets:
            if subset != test:
                train.extend(subset)
        kfolds.append((train, test))
    return kfolds


Utilizando a função criada para definir os índices de treinamento e testes a serem utilizados usando 5 folds:

In [18]:
instances = list(df.index.values)
instances

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Visualizando a tupla com os índices da validação cruzada através da função criada:

In [19]:
get_folds(list(df.index.values))

[([2, 8, 5, 6, 9, 4, 0, 1], [7, 3]),
 ([7, 3, 5, 6, 9, 4, 0, 1], [2, 8]),
 ([7, 3, 2, 8, 9, 4, 0, 1], [5, 6]),
 ([7, 3, 2, 8, 5, 6, 0, 1], [9, 4]),
 ([7, 3, 2, 8, 5, 6, 9, 4], [0, 1])]

In [20]:
# Pecorrendo a lista que contém as tuplas com os índices de treinamento e testes.
for fold in get_folds(list(df.index.values)):
     print("Indices de Treinamento:", fold[0], "Indices de Testes:", fold[1])

Indices de Treinamento: [2, 8, 5, 6, 9, 4, 0, 1] Indices de Testes: [7, 3]
Indices de Treinamento: [7, 3, 5, 6, 9, 4, 0, 1] Indices de Testes: [2, 8]
Indices de Treinamento: [7, 3, 2, 8, 9, 4, 0, 1] Indices de Testes: [5, 6]
Indices de Treinamento: [7, 3, 2, 8, 5, 6, 0, 1] Indices de Testes: [9, 4]
Indices de Treinamento: [7, 3, 2, 8, 5, 6, 9, 4] Indices de Testes: [0, 1]
